In [73]:
%pip install numpy

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.2/61.2 kB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.2/18.2 MB 36.5 MB/s eta 0:00:00 MB/s eta 0:00:01:01
Note: you may need to restart the kernel to use updated packages.


In [74]:
import os
from typing import Tuple, List, Union, Dict
from collections import defaultdict
import numpy as np
import re


# Day 1

In [25]:

def first_digit_char(s: str) -> Tuple[int, int]:
    for i, c in enumerate(s):
        if c.isdigit():
            return i,int(c)
    raise Exception(f"No digit found in {s}")

def solve(lines) -> int:
    total_sum = 0
    for line in lines:
        number = int(f"{first_digit_char(line)[1]}{first_digit_char(reversed(line))[1]}")
        total_sum += number
    return total_sum

with open("input_day_1") as f:
    total_sum = solve(f)
        
print("Total sum is", total_sum)

Total sum is 55834


## Part 2

In [27]:
digits = ("one", "two", "three", "four", "five", "six", "seven", "eight", "nine")

def first_digit_word(s: str, digits: List[str]) -> Tuple[int,int]:
    min_pair = (len(s), None)
    for i,d in enumerate(digits):
        idx = s.find(d)
        if idx != -1 and idx < min_pair[0]:
            min_pair = (idx, i+1)
    return min_pair


def solve(lines) -> int:
    total_sum = 0
    for line in lines:
        _, d1 = min(first_digit_char(line),first_digit_word(line, digits), key=lambda t: t[0])

        line_reversed = line[::-1]
        digits_reversed = [d[::-1] for d in digits]
        _, d2 = min(first_digit_char(line_reversed),first_digit_word(line_reversed, digits_reversed), key=lambda t: t[0])

        number = int(f"{d1}{d2}")
        total_sum += number
    return total_sum

with open("input_day_1") as f:
    total_sum = solve(f)

print("Total Sum", total_sum)

Total Sum 53221


# Day 2

In [30]:


def game_string_to_dict(game_str: str) -> dict[str, int]:
    game, sets = game_str.split(": ")
    game_id = int(game.split(" ")[1])
    sets = sets.split("; ")

    game_stats = defaultdict(int)
    game_stats["Game_ID"] = game_id

    for set_string in sets:
        for color in set_string.split(", "):
            v,k = color.split(" ")
            assert k in ("red", "green", "blue"), f"not in colors '{k}'"
            game_stats[k] = max(int(v), game_stats[k])
    return game_stats

id_sum = 0
with open("input_day_2") as f:
    for r in f:
        r = r.rstrip("\n")
        game = game_string_to_dict(r)
        if game["red"] <= 12 and game["green"] <= 13 and game["blue"] <= 14:
            id_sum += game["Game_ID"]
print("Total sum of filtered Game IDs", id_sum)    

Total sum of filtered Game IDs 2348


In [33]:
from functools import reduce
from operator import mul


prod_sum = 0
with open("input_day_2") as f:
    for r in f:
        r = r.rstrip("\n")
        game = game_string_to_dict(r)
        cubes = [c for c in [game["red"], game["blue"], game["green"]] if c > 0]
        prod_sum += reduce(mul, cubes, 1)
print("Total sum of products of games", prod_sum)

Total sum of products of games 76008


# Day 3

In [111]:
def parse_numbers_and_symbol_indezes(s: str) -> Tuple[Dict[Tuple[int, int], str], Dict[Tuple[int, int], str]]:
    numbers_indezes = dict() # key is inex (row, end_col), value is number
    symbol_indezes = dict() # key is index (row, col), value is symbol

    for row_idx, row in enumerate(s):
        current_number_seq = ""
        row = row.rstrip("\n")
        for i, c in enumerate(row):
            if c.isdigit():
                current_number_seq += c
            else:
                if current_number_seq:
                    numbers_indezes[(row_idx, i-1)]=current_number_seq
                    current_number_seq = ""
                if c != ".":
                    symbol_indezes[(row_idx, i)] = c
        if current_number_seq:
            numbers_indezes[(row_idx, i)]=current_number_seq
    return numbers_indezes, symbol_indezes

def compute_neighbour_symbols(numbers_indezes: Dict[Tuple[int, int], str], symbol_indezes: Dict[Tuple[int, int], str]) -> Tuple[Dict[Tuple[int, int], Tuple[str, Dict[Tuple[int, int], str]]], Dict[Tuple[int, int], Tuple[str, Dict[Tuple[int, int], str]]]]:
    numbers_with_neighbour_symbols: Dict[Tuple[int, int], Tuple[str, Dict[Tuple[int, int], str]]] = dict()
    symbols_with_neighbour_numbers: Dict[Tuple[int, int], Tuple[str, Dict[Tuple[int, int], str]]] = {
        idxs: (symbol, dict()) for idxs, symbol in symbol_indezes.items()
    }
    for (row_idx, end_index), num in sorted(numbers_indezes.items(), key=lambda t: t[0]):
        # neighbours are indezes 1 cells around the number, including diagonals
        # note that the number begins at end_index-len(num)+1
        neighbours = [(row_idx, end_index-len(num)), (row_idx, end_index+1)]
        for ic in (row_idx-1, row_idx+1):
            for j in range(end_index-len(num), end_index+2):
                neighbours.append((ic,j))

        numbers_with_neighbour_symbols[(row_idx, end_index)] = (num, dict())
        for ic,j in neighbours:
            if (ic,j) in symbol_indezes:
                numbers_with_neighbour_symbols[(row_idx, end_index)][1][(ic,j)] = symbol_indezes[(ic,j)]
                symbols_with_neighbour_numbers[(ic,j)][1][(row_idx, end_index)] = num
    return numbers_with_neighbour_symbols, symbols_with_neighbour_numbers


def compute_product_number_sum(numbers_with_neighbour_symbols: Dict[Tuple[int, int], Tuple[str, Dict[Tuple[int, int], str]]]) -> int:
    product_num_sum = 0
    for (row_idx, end_index), (num, symbols) in sorted(numbers_with_neighbour_symbols.items(), key=lambda t: t[0]):
        if len(symbols):
            print("Valid serial number", num, "at", row_idx, end_index-len(num)+1)
            product_num_sum += int(num)
        # else:
        #     print(f"No symbol found for number {num} at {(row_idx, end_index)}")
    return product_num_sum

def compute_gear_ratio_sum(symbols_with_neighbour_numbers: Dict[Tuple[int, int], Tuple[str, Dict[Tuple[int, int], str]]]) -> int:
    gear_ratio_sum = 0
    for (row_idx, col_idx), (symbol, numbers) in sorted(symbols_with_neighbour_numbers.items(), key=lambda t: t[0]):
        # if symbol is '*' and has exactly 2 neighbouring numbers, compute the gear ratio
        if symbol == "*" and len(numbers) == 2:
            gear_ratio = 1
            for num in numbers.values():
                gear_ratio *= int(num)
            gear_ratio_sum += gear_ratio
    return gear_ratio_sum

with open("input_day_3") as f:
    numbers_indezes, symbol_indezes = parse_numbers_and_symbol_indezes(f)
    numbers_with_neighbour_symbols, symbols_with_neighbour_numbers = compute_neighbour_symbols(numbers_indezes, symbol_indezes)
    print("Sum of products of numbers", compute_product_number_sum(numbers_with_neighbour_symbols))
    print("Sum of gear ratios", compute_gear_ratio_sum(symbols_with_neighbour_numbers))

Valid serial number 577 at 0 25
Valid serial number 56 at 0 90
Valid serial number 446 at 0 102
Valid serial number 793 at 0 106
Valid serial number 627 at 1 52
Valid serial number 623 at 1 76
Valid serial number 610 at 1 85
Valid serial number 16 at 1 124
Valid serial number 891 at 1 132
Valid serial number 336 at 2 21
Valid serial number 470 at 2 31
Valid serial number 84 at 2 57
Valid serial number 34 at 2 70
Valid serial number 76 at 2 95
Valid serial number 117 at 3 1
Valid serial number 359 at 3 7
Valid serial number 595 at 3 27
Valid serial number 129 at 3 42
Valid serial number 963 at 3 47
Valid serial number 722 at 3 53
Valid serial number 128 at 3 66
Valid serial number 192 at 3 77
Valid serial number 313 at 3 81
Valid serial number 31 at 3 92
Valid serial number 887 at 3 102
Valid serial number 234 at 3 120
Valid serial number 298 at 4 12
Valid serial number 922 at 4 20
Valid serial number 482 at 4 34
Valid serial number 395 at 4 97
Valid serial number 166 at 4 131
Valid ser

In [103]:
with open("input_day_3") as f:
    arr = f.readlines()

def is_symbol(row, col) -> bool:
    # check bounds
    if row < 0 or row >= len(arr) or col < 0 or col >= len(arr[row]) -1:
        return False
    # must be not digit or dot
    if not arr[row][col].isdigit() and arr[row][col] != ".":
        return True

def has_symbol_neighbour(row, col) -> bool:
    # check neighbours
    for i in range(row-1, row+2):
        for j in range(col-1, col+2):
            if is_symbol(i,j):
                return True
    return False

# find all numbers in the array
prod_sum = 0
for ir,r in enumerate(arr):
    number_seq = ""
    found_neighbour_symbol = False
    r = r.rstrip("\n")
    for ic, c in enumerate(r):
        if c.isdigit():
            number_seq += c
            if has_symbol_neighbour(ir, ic):
                found_neighbour_symbol = True
        elif number_seq:
            # terminate number sequence
            if found_neighbour_symbol:
                prod_sum += int(number_seq)
                print("Valid serial number", number_seq, "at", ir, ic-len(number_seq))
                found_neighbour_symbol = False
            # else: 
            #     print("Did not find symbol neighbour for number", number_seq, "at", (ir, ic-len(number_seq)))
            number_seq = ""
    # at end of row, check if number sequence is valid
    if number_seq and found_neighbour_symbol:
        print("Valid serial number", number_seq, "at", ir, ic-len(number_seq))
        prod_sum += int(number_seq)

print("Sum of products of numbers", prod_sum)


Valid serial number 577 at 0 25
Valid serial number 56 at 0 90
Valid serial number 446 at 0 102
Valid serial number 793 at 0 106
Valid serial number 627 at 1 52
Valid serial number 623 at 1 76
Valid serial number 610 at 1 85
Valid serial number 16 at 1 124
Valid serial number 891 at 1 132
Valid serial number 336 at 2 21
Valid serial number 470 at 2 31
Valid serial number 84 at 2 57
Valid serial number 34 at 2 70
Valid serial number 76 at 2 95
Valid serial number 117 at 3 1
Valid serial number 359 at 3 7
Valid serial number 595 at 3 27
Valid serial number 129 at 3 42
Valid serial number 963 at 3 47
Valid serial number 722 at 3 53
Valid serial number 128 at 3 66
Valid serial number 192 at 3 77
Valid serial number 313 at 3 81
Valid serial number 31 at 3 92
Valid serial number 887 at 3 102
Valid serial number 234 at 3 120
Valid serial number 298 at 4 12
Valid serial number 922 at 4 20
Valid serial number 482 at 4 34
Valid serial number 395 at 4 97
Valid serial number 166 at 4 131
Valid ser

# Day 4

In [25]:
def parse_card(card_line: str) -> int:
    segments = line.rstrip("\n").split(": ")[1].split(" | ")
    winning_numbers, my_numbers = [set([int(a) for a in s.split(" ") if a]) for s in segments]
    matching_numbers = winning_numbers.intersection(my_numbers)
    return len(matching_numbers)

score = 0
with open("input_day_4") as f:
    for line in f:
        matches = parse_card(line)
        if matches:
            score += pow(2, matches-1)
        
        
print("Score", score)

Score 20829


In [31]:
num_scratchcards = 0
multipliers = []
with open("input_day_4") as f:
    for line in f:
        matches = parse_card(line)

        card_count = (multipliers.pop(0)+1) if multipliers else 1
        num_scratchcards += card_count

        # increase next matches elements in the queue by multiplier
        # append if nessesary
        for i in range(matches):
            if i < len(multipliers):
                multipliers[i] += card_count
            else:
                multipliers.append(card_count)

        

assert len(multipliers) == 0, f"Queue should be empty {multipliers}"
print("Total number of scratchcards", num_scratchcards)

Total number of scratchcards 12648035
